In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import pymongo
import config

/home/hamoor/.local/share/virtualenvs/PennApps2018-017zYobi/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
# get data
conn = pymongo.MongoClient(config.DB_HOST, config.DB_PORT)
db = conn[config.DB_NAME]
db.authenticate(config.DB_USER, config.DB_PASS)

fake_news = db.docs.aggregate([{"$limit": db.docs.count_documents({"truth": True})}, {"$match": {"truth": False}}])
real_news = db.docs.find({"truth": True})

In [5]:
real_news = list(real_news)
fake_news = list(fake_news)

In [6]:
real_news.append(fake_news)

SyntaxError: invalid syntax (<ipython-input-6-6d8d3c231bdd>, line 1)

In [ ]:
news = real_news

In [ ]:
X, y = [], []
for x in news:
    if isinstance(x, dict):
        X.append((x['title'], x['text']))
        y.append(x['truth'])    
    else:
        for x2 in x:
            X.append((x2['title'], x2['text']))
            y.append(x2['truth'])

In [ ]:
for x in X:
    if not x[0]:
        del y[X.index(x)]
        X.remove(x)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
import preprocess as pre

c = []
for x in X_train:
    c.append((pre.tfidf_preprocess(x[0]), pre.tfidf_preprocess(x[1])))
X_train = c

c = []
for x in X_test:
    c.append((pre.tfidf_preprocess(x[0]), pre.tfidf_preprocess(x[1])))
X_test = c

In [ ]:
pipe = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', RandomForestClassifier(random_state=0))])
pipe.fit(list(x[1] for x in X_train), y_train)
predicted = pipe.predict(list(x[1] for x in X_test))

import numpy as np
np.mean(predicted == y_test)

In [ ]:
# cross validate
c = []
for x in X:
    import preprocess
    c.append(preprocess.tfidf_preprocess(x[1]))
X = c

import numpy as np

from sklearn.model_selection import StratifiedKFold
# from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier

skf = StratifiedKFold(n_splits=40)

skf.get_n_splits(X, y)

vals = []
for train_index, test_index in skf.split(X, y):
#     print('{} {}'.format(train_index, test_index))
    X_train = []
    y_train = []
    
    X_test = []
    y_test = []
    
    for ind in train_index:
        X_train.append(X[ind])
        y_train.append(y[ind])
    
    for ind in test_index:
        X_test.append(X[ind])
        y_test.append(y[ind])

    
    pipe = Pipeline([
        ('vect', CountVectorizer()), 
        ('tfidf', TfidfTransformer()), 
        ('sgd', ExtraTreesClassifier(n_estimators=100))
    ])
    pipe.fit(X_train, y_train)
    predicted = pipe.predict(X_test)
    
    print('Score: {}'.format(np.mean(predicted == y_test)))
    vals.append(np.mean(predicted == y_test))
    
print(np.mean(vals))

In [ ]:
# xgboost test
import numpy as np
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X, y)

vals = []
for train_index, test_index in skf.split(X, y):
#     print('{} {}'.format(train_index, test_index))
    X_train = []
    y_train = []
    
    X_test = []
    y_test = []
    
    for ind in train_index:
        X_train.append(X[ind])
        y_train.append(y[ind])
    
    for ind in test_index:
        X_test.append(X[ind])
        y_test.append(y[ind])

    
    pipe = Pipeline([
        ('vect', CountVectorizer()), 
        ('tfidf', TfidfTransformer()), 
        ('sgd', XGBClassifier(n_estimators=100))
    ])
    pipe.fit(X_train, y_train)
    predicted = pipe.predict(X_test)
    
    print('Score: {}'.format(np.mean(predicted == y_test)))
    vals.append(np.mean(predicted == y_test))
    
print(np.mean(vals))